# 1) Import libraries

In [2]:
from tensorflow.keras.layers import Input,Dense,Flatten,Lambda
from tensorflow.keras.models import Model
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from tensorflow.keras.models import Sequential
import numpy as np
from glob import glob
import matplotlib.pyplot as plt

# 2) Re-size all images to standard size used in VGG

In [3]:
IMAGE_SIZE = [224,224]

# 3) Provide the dataset path

In [5]:
train_path = 'C:/Users/sameer/Downloads/Datasets/Datasets/Train'
test_path = 'C:/Users/sameer/Downloads/Datasets/Datasets/Test'

# 4) Load the pretrained model

In [6]:
vgg = VGG16(input_shape=IMAGE_SIZE+[3], weights='imagenet',include_top=False)

58892288/58889256 [==============================] - 11s 0us/step


In [8]:
vgg.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

# 5) Don't train the existing weights. We are using the pretrained model of VGG which was already trained on Imagenet dataset so we will use those weights only. We wont re train those weights.

In [9]:
vgg.layers   #shows 16 layers objects

In [10]:
for layer in vgg.layers:
    layer.trainable=False

# 6) As we know we remove bottom and top layer of VGG using include_top=false since it was trained for 1000 categories nd we have just 3. So use glob to state our 3 categories and attach it to the pretraine model

In [13]:
folders=glob('C:/Users/sameer/Downloads/Datasets/Datasets/Train/*')

In [14]:
folders

['C:/Users/sameer/Downloads/Datasets/Datasets/Train\\audi',
 'C:/Users/sameer/Downloads/Datasets/Datasets/Train\\lamborghini',
 'C:/Users/sameer/Downloads/Datasets/Datasets/Train\\mercedes']

In [15]:
vgg.output

<tf.Tensor 'block5_pool/MaxPool:0' shape=(None, 7, 7, 512) dtype=float32>

# 7) Now we have the flatten the vgg output

In [16]:
x=Flatten()(vgg.output)

# 8) Now put this flattened array to the dense network and then create the model object where inputs and outputs will be two parameters..outputs=predictions and inputs =vgg.input(imagesize)

In [20]:
prediction = Dense(len(folders),activation='softmax')(x)

In [21]:
#create model object

model = Model(inputs=vgg.input,outputs=prediction)

In [22]:
model.summary()   #you can see now we have added 3 nodes as the last layer

Model: "functional_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)      

# 9) Tell the model what optimization and loss u want to use

In [23]:
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

# 10) Now use image data generator to import the images from dataset

In [24]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [25]:
train_datagen = ImageDataGenerator(rescale=1./255,shear_range = 0.2, zoom_range=0.2, horizontal_flip=True)

In [26]:
test_datagen=ImageDataGenerator(rescale=1./255)

In [27]:
training_set=train_datagen.flow_from_directory('C:/Users/sameer/Downloads/Datasets/Datasets/Train',target_size=(224,224),batch_size=32,class_mode='categorical')

Found 64 images belonging to 3 classes.


In [28]:
test_set=test_datagen.flow_from_directory('C:/Users/sameer/Downloads/Datasets/Datasets/Test',target_size=(224,224),batch_size=32,class_mode='categorical')

Found 58 images belonging to 3 classes.


In [29]:
r=model.fit(training_set, validation_data=test_set,epochs=20,steps_per_epoch=len(training_set),validation_steps=len(test_set))

Epoch 1/20
2/2 [==============================] - 47s 24s/step - loss: 1.8554 - accuracy: 0.3125 - val_loss: 1.9971 - val_accuracy: 0.3276
Epoch 2/20
2/2 [==============================] - 48s 24s/step - loss: 1.5355 - accuracy: 0.4219 - val_loss: 2.1005 - val_accuracy: 0.1724
Epoch 3/20
2/2 [==============================] - 52s 26s/step - loss: 1.2570 - accuracy: 0.5312 - val_loss: 1.3137 - val_accuracy: 0.5517
Epoch 4/20
2/2 [==============================] - 43s 21s/step - loss: 0.7921 - accuracy: 0.7188 - val_loss: 1.1184 - val_accuracy: 0.6034
Epoch 5/20
2/2 [==============================] - 40s 20s/step - loss: 0.5787 - accuracy: 0.7812 - val_loss: 0.8051 - val_accuracy: 0.6897
Epoch 6/20
2/2 [==============================] - 39s 19s/step - loss: 0.6552 - accuracy: 0.7188 - val_loss: 0.7725 - val_accuracy: 0.5862
Epoch 7/20
2/2 [==============================] - 38s 19s/step - loss: 0.3286 - accuracy: 0.8281 - val_loss: 0.5840 - val_accuracy: 0.7931
Epoch 8/20
2/2 [===========

In [30]:
y_pred=model.predict(test_set)

In [31]:
y_pred   #3 values are the probability of being audi, lamborhini or mercedes..alphabet wise

array([[9.38500166e-01, 3.29882751e-04, 6.11699745e-02],
       [4.41167923e-03, 8.47990155e-01, 1.47598103e-01],
       [1.70403630e-01, 3.45090270e-01, 4.84506100e-01],
       [4.47861930e-05, 9.96966422e-01, 2.98872613e-03],
       [8.01068381e-04, 9.97657180e-01, 1.54163281e-03],
       [2.63235602e-03, 9.58827496e-01, 3.85402031e-02],
       [6.45751506e-03, 9.75249231e-01, 1.82932690e-02],
       [7.64019907e-01, 4.94617643e-03, 2.31033906e-01],
       [6.11934721e-01, 2.71673203e-01, 1.16392098e-01],
       [9.87760782e-01, 7.72445404e-04, 1.14667462e-02],
       [4.34182817e-04, 9.94072974e-01, 5.49282134e-03],
       [3.79994838e-03, 8.72737586e-01, 1.23462535e-01],
       [4.71872569e-04, 9.98212934e-01, 1.31523365e-03],
       [2.67204084e-02, 3.22972797e-02, 9.40982342e-01],
       [2.04627011e-02, 1.13415845e-01, 8.66121471e-01],
       [1.59044731e-02, 9.77779552e-03, 9.74317789e-01],
       [1.55046672e-01, 7.04308785e-03, 8.37910295e-01],
       [3.18483621e-01, 4.59936

In [32]:
import numpy as np

In [33]:
#apply argmax and take that index which has the highest value for that record

y_pred=np.argmax(y_pred,axis=1)

In [34]:
y_pred   #output of test data

array([0, 1, 2, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 2, 2, 2, 2, 2, 2, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 2, 1, 2, 1, 2, 2, 0, 1, 0, 2, 1, 1, 1, 0, 2,
       1, 1, 1, 1, 2, 2, 1, 2, 2, 1, 2, 2, 1, 1], dtype=int64)

# Reading image as input

In [35]:
from tensorflow.keras.preprocessing import image

In [36]:
img=image.load_img('C:/Users/sameer/Downloads/Datasets/Datasets/Train/audi/1.jpg',target_size=(224,224))

In [37]:
x=image.img_to_array(img)
x

array([[[233., 219., 208.],
        [234., 220., 209.],
        [237., 223., 212.],
        ...,
        [213., 216., 221.],
        [218., 221., 226.],
        [161., 164., 169.]],

       [[240., 226., 215.],
        [240., 226., 215.],
        [243., 229., 218.],
        ...,
        [213., 216., 221.],
        [217., 220., 225.],
        [161., 164., 169.]],

       [[247., 233., 222.],
        [247., 233., 222.],
        [250., 236., 225.],
        ...,
        [213., 216., 221.],
        [217., 220., 225.],
        [161., 164., 169.]],

       ...,

       [[130., 112., 108.],
        [135., 117., 113.],
        [138., 124., 123.],
        ...,
        [213., 212., 208.],
        [215., 210., 207.],
        [215., 210., 207.]],

       [[126., 108., 104.],
        [133., 115., 111.],
        [130., 116., 115.],
        ...,
        [217., 216., 212.],
        [218., 213., 210.],
        [218., 213., 210.]],

       [[121., 103.,  99.],
        [133., 115., 111.],
        [135., 1

In [38]:
x.shape

(224, 224, 3)

In [40]:
x=x/255
x

array([[[0.00358324, 0.00336794, 0.00319877],
        [0.00359862, 0.00338331, 0.00321415],
        [0.00364475, 0.00342945, 0.00326028],
        ...,
        [0.00327566, 0.0033218 , 0.00339869],
        [0.00335256, 0.00339869, 0.00347559],
        [0.00247597, 0.00252211, 0.002599  ]],

       [[0.00369089, 0.00347559, 0.00330642],
        [0.00369089, 0.00347559, 0.00330642],
        [0.00373702, 0.00352172, 0.00335256],
        ...,
        [0.00327566, 0.0033218 , 0.00339869],
        [0.00333718, 0.00338331, 0.00346021],
        [0.00247597, 0.00252211, 0.002599  ]],

       [[0.00379854, 0.00358324, 0.00341407],
        [0.00379854, 0.00358324, 0.00341407],
        [0.00384468, 0.00362937, 0.00346021],
        ...,
        [0.00327566, 0.0033218 , 0.00339869],
        [0.00333718, 0.00338331, 0.00346021],
        [0.00247597, 0.00252211, 0.002599  ]],

       ...,

       [[0.00199923, 0.00172241, 0.0016609 ],
        [0.00207612, 0.00179931, 0.00173779],
        [0.00212226, 0

In [41]:
from tensorflow.keras.applications.resnet50 import preprocess_input

In [42]:
x=np.expand_dims(x,axis=0)
img_data=preprocess_input(x)
img_data.shape

(1, 224, 224, 3)

In [43]:
model.predict(img_data)

array([[1.7535434e-05, 1.1717037e-06, 9.9998128e-01]], dtype=float32)

In [48]:
a=np.argmax(model.predict(img_data),axis=1)
a

array([2], dtype=int64)